In [2]:
%run ../../../utils/commonImports.py
%run ../../../utils/tradingImports.py
%matplotlib inline

from base import supres
from exputils import *

C:\Users\david.vesely\AppData\Local\conda\conda\envs\crypto36\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Load data

In [3]:
dropbox_dir = 'D:\\Dropbox\\My work\\krypl-project'
# dropbox_dir = '/Users/david.vesely/Dropbox/My work/krypl-project'
features = ['support']
pairs = read_tsv('selected-pairs.tsv', header=None)[0].tolist()

data_dict = {p: read_all(p, dropbox_dir) for p in pairs}
supports = read_tsv('labeled.tsv')

In [4]:
window_sizes = [4]
shifts = [1, 2, 3]

DAY = 48
max_hold_time = 5 * DAY
stop_loss = 0.05
fee = 0.0025

# Support changes

In [5]:
def get_changes(data, shift=1):
    support_vals = data['support'].values
    size = support_vals.shape[0]
    changes = (support_vals[shift:] / support_vals[:size-shift]) - 1
    for _ in range(shift):
        changes = np.insert(changes, 0, 0)
    data[f'change_{shift}'] = changes
    return data

In [6]:
from tqdm import tqdm_notebook

n = len(pairs) * len(window_sizes) * len(shifts)
with tqdm_notebook(total=n) as pbar:
    supports_data = pd.DataFrame()
    for pair in pairs:
        for win_size in window_sizes:
            supports_df = supports.query(f'pair == "{pair}"')\
                    .query(f'win_size == {win_size}')\
                    .sort_values('date')\
                    .copy()
            for shift in shifts:
                supports_df = get_changes(supports_df, shift)
                pbar.update()
                
            supports_data = supports_data.append(supports_df)
            

# Consolidate label for better learning

In [7]:
def decision(r):
    if r['max_profit'] >= -r['change_1']:
        return -r['change_1']
    else:
        return r['max_profit']

supports_data['max_profit_consolidated'] = supports_data.apply(lambda r: decision(r), axis=1)

In [9]:
write_tsv(supports_data, 'featured.tsv')